In [27]:
#Sistemas de recomendação

1) Carregar os dados
2) Pré-processamento dos dados
3) Divisão dos dados entre treinamento e teste
4) Criar o sistema de recomendação através do ALS
5) Realizar previsões
6) Testar o modelo

In [28]:
#importa a biblioteca que cria a seção do spark
from pyspark.sql import SparkSession 

In [29]:
#inicia a seção para a utilização do spark
spark = SparkSession\
    .builder\
    .appName("recomendacao")\
    .getOrCreate() #cria a seção caso não exista ou obtém a já criada    

In [30]:
#define o diretório que contém o arquivo a ser utilizado
diretorioRecomendacao="./data/u.data"

1) Carregando o arquivo.

In [31]:
#lendo arquivos armazenados através da função genérica
#formato .data não pode salvar direto em dataframe, por isso usa-se rdd
rdd_movies = spark.sparkContext.textFile(diretorioRecomendacao)

In [32]:
#mostra 10 primeiros linhas
# user id | item id | rating | timestamp
rdd_movies.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [33]:
#definindo as bibliotecas a serem utilizadas
#MLlib utilizada para implementar os algoritmos ALS e Rating
from pyspark.mllib.recommendation import ALS, Rating  

2) Pré-processamento dos dados
3) Separando os dados para treinamento e teste

In [34]:
#dividindo os dados entre treinamento e teste
(trainRatings, testRatings) = rdd_movies.randomSplit([0.7, 0.3])

In [35]:
#mostra 5 primeiros linhas do RDD de treino
trainRatings.take(5)

['186\t302\t3\t891717742',
 '244\t51\t2\t880606923',
 '298\t474\t4\t884182806',
 '253\t465\t5\t891628467',
 '6\t86\t3\t883603013']

In [36]:
#print da primeira linha do RDD d teste
testRatings.first()  

'196\t242\t3\t881250949'

In [37]:
#aplicando a função Rating
#mapeia o RDD separando pelo tab (\t) e aplica a função rating nas colunas 0,1,2
trainingData = trainRatings.map(lambda l: l.split('\t')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2]))) 

In [38]:
#print do RDD criado através da função Rating
trainingData.first()

Rating(user=186, product=302, rating=3.0)

In [39]:
#mesmo procedimento para os dados de teste (pegando apenas id do usuário e do filme)
testData = testRatings.map(lambda l: l.split('\t')).map(lambda l: (int(l[0]), int(l[1])))

In [40]:
#id do usuário | id do filme
testData.first()

(196, 242)

4) Construindo o modelo ALS

In [41]:
#definindo as variáveis do modelo

#número de fatores latentes do modelo  
# R->P (usuários)*Q (itens) => R_mxn = P_mxrank * Q_rankxm  (em que m= numero de usuário e n= numero de itens)
#quantidade de características de cada um dos elementos (usuários e itens)
rank = 10  

#número de iterações realizadas pelo modelo para preencher números faltosos da matriz
numIterations = 50 

#treina o modelo
model = ALS.train(trainingData, rank, numIterations) 

5) Previsão do modelo

In [42]:
#entrada (usuário,produto)
model.predict(253, 465) 

3.993117039622087

In [43]:
#previsão para todos os dados de teste
previsao = model.predictAll(testData)  
previsao.first()

Rating(user=894, product=1080, rating=2.8083768225200822)

In [44]:
#mapea para a exibição
previsao = previsao.map(lambda l: ((l[0], l[1]), l[2])) 
previsao.take(5)

[((894, 1080), 2.8083768225200822),
 ((368, 320), 4.579678877165637),
 ((264, 320), 6.015349531739346),
 ((896, 320), 6.241397340684338),
 ((592, 320), 3.6001377424801526)]

In [45]:
#mapea para exibição e utilização na análise
testRating2 = testRatings.map(lambda l: l.split('\t')).map(lambda l: ((int(l[0]), int(l[1])), float(l[2]))) 

In [46]:
testRating2.first()

((196, 242), 3.0)

In [48]:
ratingsAndPredictions = testRating2.join(previsao)
ratingsAndPredictions.take(5)

#usuário | filme | nota real | nota prevista

[((196, 242), (3.0, 3.3171186144071223)),
 ((305, 451), (3.0, 2.811049936167865)),
 ((160, 234), (5.0, 4.36807447112667)),
 ((225, 193), (4.0, 4.851721204119223)),
 ((293, 471), (3.0, 2.9090834011054216))]

#6) Avaliação do modelo

In [49]:
MSE = ratingsAndPredictions.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print ("Erro médio quadrado = " + str(MSE))

Erro médio quadrado = 1.3400298710922356


In [50]:
#dado um usuário -> recomenda os top 5 produtos
model.recommendProducts(105, 5) 

[Rating(user=105, product=1278, rating=6.7075280489592375),
 Rating(user=105, product=1137, rating=6.6448247127902835),
 Rating(user=105, product=36, rating=6.623917573807324),
 Rating(user=105, product=980, rating=6.609421457372327),
 Rating(user=105, product=967, rating=6.437895729599492)]

In [51]:
#dado um produto -> recomenda top 5 usuários
model.recommendUsers(1, 5)  #filme Toy Story (1995)

[Rating(user=427, product=1, rating=7.107117287366402),
 Rating(user=404, product=1, rating=6.153278357749934),
 Rating(user=887, product=1, rating=6.060045635622167),
 Rating(user=519, product=1, rating=6.009313343148387),
 Rating(user=98, product=1, rating=5.952581551486009)]

In [52]:
#mostrando o vetor de características referentes usuários (coluna - P)
model.userFeatures().take(1)[0]

(8,
 array('d', [-0.9412178993225098, -0.5234482288360596, 0.7469764947891235, -1.0941760540008545, 0.6517865061759949, 0.3931030333042145, -0.09774608165025711, 0.39126020669937134, 0.8421290516853333, 0.06971098482608795]))

In [53]:
#mostra o vetor de características referente a um produto (linha - Q)
model.productFeatures().take(1)[0]

(8,
 array('d', [-1.4833488464355469, -1.1450612545013428, 0.3274649679660797, -0.16559606790542603, -0.22634750604629517, -0.027587857097387314, 0.431527316570282, 0.9530938863754272, 1.6408685445785522, 0.3619742691516876]))